<a href="https://colab.research.google.com/github/Niteen01/Fake-news-detection-using-ML-/blob/main/FakeNewsDetection_using_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from nltk.corpus import stopwords
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from nltk.stem.porter import PorterStemmer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras import backend
# from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, LSTM

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

import nltk

In [ ]:
pip install transformers

In [ ]:
nltk.download('stopwords')
news_dataset = pd.read_csv("/content/drive/MyDrive/dataset/train.csv")
news_dataset.head()
news_dataset.isnull().sum()
news_dataset = news_dataset.fillna('')
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']
port_stem = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def removePunchuation(content):
    new_String = re.sub('[^a-zA-Z]',' ',str(content))
    return new_String

def convertToLower(content):
    content=content.lower()
    content = content.split()
    return content


def removeStopWord(content):
    temp=[]
    for word in content:
        if not word in stopwords.words('english'):
            temp.append(word)
    return temp

def stemming(content):
    temp=[]
    for word in content:
        str=port_stem.stem(word)
        temp.append(str)
    return temp

def combineContent(content):
    content = ' '.join(content)
    return content

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(removePunchuation)
news_dataset['content'] = news_dataset['content'].apply(convertToLower)
news_dataset['content'] = news_dataset['content'].apply(removeStopWord)
news_dataset['content'] = news_dataset['content'].apply(stemming)
news_dataset['content'] = news_dataset['content'].apply(combineContent)


In [ ]:
X = news_dataset['content'].values
Y = news_dataset['label'].values

ERT = news_dataset['content'].values
ERT_value = news_dataset['label'].values

vectorizer = TfidfVectorizer()

vectorizer.fit(X)
X = vectorizer.transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

In [ ]:
# 1. logistics regression
model = LogisticRegression()
scores = cross_val_score(model, X, Y, cv=10)
print("logistic regression")
print("Cross-Validation Scores:", scores)
print("Mean 10 cv:", np.mean(scores))

model.fit(X_train, Y_train)
X_train_prediction = model.predict(X_train)
X_new = X_test[4]
prediction = model.predict(X_new)

# if (prediction[0]==0):
#   print('The news is Real')
# else:
#   print('The news is Fake')

y_pred = model.predict(X_test)

f1 = f1_score(Y_test, y_pred, average='weighted')
print("F1 score: ", f1)

precision = precision_score(Y_test, y_pred)
print('Precision score: ', precision)

recall = recall_score(Y_test, y_pred)
print('Recall score: ', recall)

accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)

logistic regression
Cross-Validation Scores: [0.97932692 0.97692308 0.97692308 0.97596154 0.98221154 0.97740385
 0.97932692 0.97788462 0.97596154 0.97884615]
Mean 10 cv: 0.9780769230769231
F1 score:  0.9790819028111927
Precision score:  0.9659197012138189
Recall score:  0.9932789246279404
Accuracy: 0.9790865384615385


In [ ]:
# 2. support vector machine
svm = SVC(kernel='linear', C=1.0, random_state=42)
scores = cross_val_score(svm, X, Y, cv=10)
print("svc")
print("Cross-Validation Scores:", scores)
print("Mean 10 cv:", np.mean(scores))

svm.fit(X_train, Y_train)
y_pred = svm.predict(X_test)

f1 = f1_score(Y_test, y_pred, average='weighted')
print("F1 score: ", f1)

precision = precision_score(Y_test, y_pred)
print('Precision score: ', precision)

recall = recall_score(Y_test, y_pred)
print('Recall score: ', recall)

accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy: ", accuracy)

svc
Cross-Validation Scores: [0.99230769 0.99326923 0.9875     0.99326923 0.99471154 0.99375
 0.99230769 0.99567308 0.98798077 0.99086538]
Mean 10 cv: 0.9921634615384616
F1 score:  0.9913461338436202
Precision score:  0.9904168663152851
Recall score:  0.9923187710033605
Accuracy:  0.9913461538461539


In [ ]:
# 3. decision tree
dtc = DecisionTreeClassifier(random_state=42)
scores = cross_val_score(dtc, X, Y, cv=10)
print("decision tree")
print("Cross-Validation Scores:", scores)
print("Mean 10cv:", np.mean(scores))
dtc.fit(X_train, Y_train)

# Test the Decision Tree model and print accuracy score
y_pred = dtc.predict(X_test)

f1 = f1_score(Y_test, y_pred, average='weighted')
print("F1 score: ", f1)

precision = precision_score(Y_test, y_pred)
print('Precision score: ', precision)

recall = recall_score(Y_test, y_pred)
print('Recall score: ', recall)

accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)

decision tree
Cross-Validation Scores: [0.99615385 0.99471154 0.99182692 0.99519231 0.99423077 0.99471154
 0.99567308 0.99230769 0.99375    0.99134615]
Mean 10cv: 0.9939903846153847
F1 score:  0.9918269155204523
Precision score:  0.9937349397590362
Recall score:  0.9899183869419107
Accuracy: 0.9918269230769231


In [ ]:
# 4. knn
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
scores = cross_val_score(knn, X, Y, cv=10)
print("knn")
print("Cross-Validation Scores:", scores)
print("Mean F1 Score:", np.mean(scores))
knn.fit(X_train, Y_train)
y_pred = knn.predict(X_test)

f1 = f1_score(Y_test, y_pred, average='weighted')
print("F1 score: ", f1)

precision = precision_score(Y_test, y_pred)
print('Precision score: ', precision)

recall = recall_score(Y_test, y_pred)
print('Recall score: ', recall)

accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)

knn
Cross-Validation Scores: [0.52788462 0.53028846 0.53028846 0.52211538 0.52451923 0.52019231
 0.52307692 0.52740385 0.52692308 0.52692308]
Mean F1 Score: 0.5259615384615384
F1 score:  0.38247842715434816
Precision score:  0.5122970978848992
Recall score:  1.0
Accuracy: 0.5233173076923077


In [ ]:
# 5. naive baiyes
nb = MultinomialNB()
scores = cross_val_score(nb, X, Y, cv=10)
print("naive bayes")
print("Cross-Validation Scores:", scores)
print("Mean F1 Score:", np.mean(scores))
nb.fit(X_train, Y_train)
y_pred = nb.predict(X_test)

f1 = f1_score(Y_test, y_pred, average='weighted')
print("F1 score: ", f1)

precision = precision_score(Y_test, y_pred)
print('Precision score: ', precision)

recall = recall_score(Y_test, y_pred)
print('Recall score: ', recall)

accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)

naive bayes
Cross-Validation Scores: [0.95625    0.95576923 0.95961538 0.95336538 0.95769231 0.95817308
 0.95240385 0.95817308 0.95048077 0.95096154]
Mean F1 Score: 0.9552884615384615
F1 score:  0.9549831652908682
Precision score:  0.9932362122788762
Recall score:  0.9164666346615459
Accuracy: 0.9550480769230769
